In [1]:
print('Importing...')

import glob
import os
import numpy as np
import time as systime
import re 
import textgrid as tg

from scipy.stats import multivariate_normal

from bokeh.plotting import show
from bokeh.io import output_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from ipyparallel import Client
import _pickle as pickle

import sys

print('success')

Importing...
success


- Manually specify a top string, a bottom string, and an MFCC output.

In [2]:
top_string = [0,1,0]
top_strings = [top_string]
bot_string = [0,2,1,0]
bot_string_states = [0,0,1,2, 6,7,7,8, 3,3,4,5, 0,1,2,2]
print('success')

success


Let's define the HMM/GMM parameters

In [3]:
# transition = [[.5, .5, 0],
#              [0, .5, .5],
#              [.75, 0, .25]]

components = []
# Distribution over components for each bottom PLU state
components.extend([[.7,.3],
             [.9,.1],
             [.3,.7]])
components.extend([[.3,.7],
             [.5,.5],
             [.8,.2]])
components.extend([[.4,.6],
             [.2,.8],
             [.9,.1]])

# We need a set of parameters for each component
# for each HMM state for each bottom PLU
# So in this case, we need 2 * 3 * 3 = 18 sets of parameters

component_parameters = [
        [
            [[0,0], [[1,0],[0,1]]], # bottom PLU 0, HMM state 0, component 0
            [[0.1,0.1], [[1,0],[0,1]]]  # bottom PLU 0, HMM state 0, component 1
        ],
        [
             [[0.2,0.2], [[1,0],[0,1]]], # bottom PLU 0, HMM state 1, component 0
             [[0.3,0.3], [[1,0],[0,1]]]  # bottom PLU 0, HMM state 1, component 1
        ],
        [
            [[0.4,0.4], [[1,0],[0,1]]], # bottom PLU 0, HMM state 2, component 0
            [[0.5,0.5], [[1,0],[0,1]]]  # bottom PLU 0, HMM state 2, component 1
        ],
    
    
        [
            [[1.0,1.0], [[1,0],[0,1]]], # bottom PLU 1, HMM state 0, component 0
            [[1.1,1.1], [[1,0],[0,1]]]  # bottom PLU 1, HMM state 0, component 1
        ],
        [
             [[1.2,1.2], [[1,0],[0,1]]], # bottom PLU 1, HMM state 1, component 0
             [[1.3,1.3], [[1,0],[0,1]]]  # bottom PLU 1, HMM state 1, component 1
        ],
        [
            [[1.4,1.4], [[1,0],[0,1]]], # bottom PLU 1, HMM state 2, component 0
            [[1.5,1.5], [[1,0],[0,1]]]  # bottom PLU 1, HMM state 2, component 1
        ],
    
          
        [
            [[2.0,2.0], [[1,0],[0,1]]], # bottom PLU 2, HMM state 0, component 0
            [[2.1,2.1], [[1,0],[0,1]]]  # bottom PLU 2, HMM state 0, component 1
        ],
        [
             [[2.2,2.2], [[1,0],[0,1]]], # bottom PLU 2, HMM state 1, component 0
             [[2.3,2.3], [[1,0],[0,1]]]  # bottom PLU 2, HMM state 1, component 1
        ],
        [
            [[2.4,2.4], [[1,0],[0,1]]], # bottom PLU 2, HMM state 2, component 0
            [[2.5,2.5], [[1,0],[0,1]]]  # bottom PLU 2, HMM state 2, component 1
        ]
    

]


densities = [ [multivariate_normal(mean=comp[0], cov=comp[1]) for comp in comps] for comps in component_parameters]

densities_flat = []

[densities_flat.extend(x) for x in densities]

print('success')

print(densities_flat)


success
[<scipy.stats._multivariate.multivariate_normal_frozen object at 0x109871da0>, <scipy.stats._multivariate.multivariate_normal_frozen object at 0x109871f60>, <scipy.stats._multivariate.multivariate_normal_frozen object at 0x109871f98>, <scipy.stats._multivariate.multivariate_normal_frozen object at 0x109876128>, <scipy.stats._multivariate.multivariate_normal_frozen object at 0x1098761d0>, <scipy.stats._multivariate.multivariate_normal_frozen object at 0x109876278>, <scipy.stats._multivariate.multivariate_normal_frozen object at 0x109876320>, <scipy.stats._multivariate.multivariate_normal_frozen object at 0x1098763c8>, <scipy.stats._multivariate.multivariate_normal_frozen object at 0x109876470>, <scipy.stats._multivariate.multivariate_normal_frozen object at 0x109876518>, <scipy.stats._multivariate.multivariate_normal_frozen object at 0x1098765c0>, <scipy.stats._multivariate.multivariate_normal_frozen object at 0x109876668>, <scipy.stats._multivariate.multivariate_normal_frozen o

In [4]:
print('running...')

def sample_gmm(bottom_hmm_states):
    
    mfccs = []
    
    for state in bottom_hmm_states:
        vec = None
        gmm_component_choice = np.random.choice(2,p=components[state])
        vec = np.random.multivariate_normal(*component_parameters[state][gmm_component_choice])
        mfccs.append(vec)
            
    return np.array(mfccs)

all_data = []


mfccs = sample_gmm(bot_string_states)

all_data.append(mfccs)

print('success')
        
print('bottom string: \n', bot_string_states)

print('mfccs: \n', mfccs)


for frame in mfccs:
    print([np.log(mv.pdf(frame)) for mv in densities_flat])
        

running...
success
bottom string: 
 [0, 0, 1, 2, 6, 7, 7, 8, 3, 3, 4, 5, 0, 1, 2, 2]
mfccs: 
 [[-0.12451349 -0.43472948]
 [ 0.81867572 -0.42454106]
 [-0.33840169 -0.10171253]
 [ 1.17512062 -0.17312668]
 [ 4.03781935  1.34149782]
 [ 2.95664307  1.76554284]
 [ 2.78996405  2.24708973]
 [ 2.8359131   1.51149785]
 [ 0.72764319  1.91596981]
 [ 1.57475266  0.77958692]
 [ 1.00761908 -0.81206207]
 [-0.43196912  1.76778703]
 [ 0.20876681  0.56040988]
 [-0.68616729 -0.47756926]
 [-0.50238189 -0.27585305]
 [-0.28872596 -0.12660419]]
[-1.940123729621849, -2.0060480261090197, -2.0919723225961908, -2.1978966190833615, -2.3238209155705327, -2.4697452120577039, -3.4993666944935584, -3.7652909909807297, -4.051215287467901, -4.3571395839550719, -4.6830638804422424, -5.0289881769294134, -7.0586096593652679, -7.5245339558524389, -8.0104582523396104, -8.5163825488267797, -9.0423068453139521, -9.5882311418011223]
[-2.2631095861348487, -2.2336961203677701, -2.2242826546006911, -2.2348691888336125, -2.26545572

In [5]:
def collect_data_stats(data):
    """Job to collect the statistics."""
    # We  re-import this module here because this code will run
    # remotely.
    
    stats_0 = data.shape[0]
    stats_1 = data.sum(axis=0)
    stats_2 = (data**2).sum(axis=0)
    retval = (
        stats_0,
        stats_1,
        stats_2
    )
    return retval

data_stats = list(map(collect_data_stats, all_data))

print('success')

success


In [6]:

def accumulate_stats(data_stats):
    n_frames = data_stats[0][0]
    mean = data_stats[0][1]
    var = data_stats[0][2]
    for stats_0, stats_1, stats_2 in data_stats[1:]:
        n_frames += stats_0
        mean += stats_1
        var += stats_2
    mean /= n_frames
    var = (var / n_frames) - mean**2

    data_stats = {
        'count': n_frames,
        'mean': mean,
        'var': var
    }
    return data_stats

final_data_stats = accumulate_stats(data_stats)

print('success')

success


In [7]:
print("Running...")

import subprocess
import amdtk
num_tops = 4

print("Finished imports in ipython notebook.")

print(len(all_data))
print(len(top_strings))
all_data_and_tops = list(zip(all_data, top_strings))
#print(all_data_and_tops[0])

print("starting engines")
subprocess.Popen(['ipcluster', 'start',' --profile', 'default',' -n', '4', '--daemonize'])
subprocess.Popen(['sleep', '10']).communicate()


rc = Client(profile='default')
dview = rc[:]
print('Connected to', len(dview), 'jobs.')


elbo = []
time = []
def callback(args):
    elbo.append(args['lower_bound'])
    time.append(args['time'])
    print('elbo=' + str(elbo[-1]), 'time=' + str(time[-1]))
 

print("Creating phone loop model...")
conc = 0.1



model = amdtk.PhoneLoopNoisyChannel.create(
    n_units=5,  # number of acoustic units
    n_states=3,   # number of states per unit
    n_comp_per_state=3,   # number of Gaussians per emission
    n_top_units=num_tops, # size of top PLU alphabet
    mean=np.zeros_like(final_data_stats['mean']), 
    var=np.ones_like(final_data_stats['var']),
    max_slip_factor=.05
    #concentration=conc
)


print("Creating VB optimizer...")   
optimizer = amdtk.ToyNoisyChannelOptimizer(
    dview, 
    final_data_stats, 
    args= {'epochs': 10,
     'batch_size': 4,
     'lrate': 0.01,
     'pkl_path': "example_test",
     'log_dir': 'logs'},
    model=model,
    dir_path='dir'
)

print("Running VB optimization...")
begin = systime.time()
print("running with toy data")
print('len(all_data_and_tops[0])')
print(len(all_data_and_tops[0]))
optimizer.run(all_data_and_tops, callback)
end = systime.time()
print("VB optimization took ",end-begin," seconds.")

# fig1 = figure(
#     x_axis_label='time (s)', 
#     y_axis_label='ELBO',
#     width=400, 
#     height=400
# )
# x = np.arange(0, len(elbo), 1)
# fig1.line(x, elbo)
#show(fig1)

print("\nDECODING\n")

date_string = systime.strftime("textgrids_%Y-%m-%d_%H:%M")

# Need to change this according to 
samples_per_sec = 100

all_data_and_tops = list(zip(all_data, top_strings))

for data, tops in all_data_and_tops:

    # Normalize the data
    data_mean = np.mean(data)
    data_var = np.var(data)
    data = (data-data_mean)/np.sqrt(data_var)

    #result = model.decode(data, tops, state_path=False)
    #result_path = model.decode(data, tops, state_path=True)
    (result_intervals, edit_path) = model.decode(data, tops, phone_intervals=True, edit_ops=True)

    print("---")
    print("Phone sequence:")
    print(result_intervals)
    print(edit_path)
    

Running...
INSIDE INIT
SUCCESSFUL IMPORT IN INIT
Finished imports in ipython notebook.
1
1
starting engines
Connected to 4 jobs.
Creating phone loop model...
[-2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825
 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825]
[-2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825
 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825]
[-2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825
 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825]
[-2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825
 -2.92896825 -2.92896825 -2.92896825 -2.92896825 -2.92896825]
Creating VB optimizer...
importing numpy on engine(s)
importing read_htk from amdtk on engine(s)
importing _pickle on engine(s)
importing os on engine(s)
Running VB optimization...
running with toy data
len(all_data_and_tops[0])
2
[[  3.38517447e-02   9.18401567e-03   5.65481677e-02]
 [  6.33967480e-0

[[  3.38227915e-02   9.60319688e-03   5.60920016e-02]
 [  6.26695199e-02   1.67180977e-02   1.85673478e-02]
 [  8.70562825e-02   2.85202518e-03   2.98032979e-02]
 [  1.89187529e-01   1.14359086e-01   2.25916592e-01]
 [  2.88162087e-01   2.45816376e-01   7.75785729e-02]
 [  8.53964321e-02   1.15261536e-01   2.92891112e-01]
 [  4.16427393e-01   4.83975469e-01   2.90701313e-01]
 [  4.65490185e-01   4.25108933e-01   3.30704417e-01]
 [  4.29151361e-01   4.53302339e-01   2.45474401e-01]
 [  4.49429649e-01   4.49844064e-01   9.44383341e-01]
 [  1.50050261e+00   1.16297908e+00   2.92674636e+00]
 [  2.81277610e-01   5.42470955e-01   1.31785260e+00]
 [  1.52057784e-01   5.46505338e-02   7.20961389e-02]
 [  1.66920540e-01   2.54154353e-02   8.23459074e-02]
 [  3.94004712e-02   1.33607590e-01   2.05929626e-01]]
[[  7.68575713e-03   5.02654648e-02  -9.15205137e-03   2.58830173e-02
    3.38227915e-02   3.38227915e-02   3.38227915e-02   3.38227915e-02]
 [  4.13109501e-03   2.59653952e-03  -4.36290414

[[ 0.03381116  0.01043024  0.0552621 ]
 [ 0.06135689  0.01753482  0.01935346]
 [ 0.08486828  0.00344804  0.03010277]
 [ 0.18296376  0.11202796  0.21716411]
 [ 0.27685686  0.23619822  0.07575096]
 [ 0.08363261  0.1139491   0.28394457]
 [ 0.41209964  0.48077192  0.28689372]
 [ 0.46186932  0.42408219  0.3286644 ]
 [ 0.42095045  0.45674222  0.25559355]
 [ 0.44764773  0.445933    0.89432507]
 [ 1.46389087  1.11601725  3.13044055]
 [ 0.29558936  0.55343305  1.31915984]
 [ 0.14600498  0.05303091  0.07323671]
 [ 0.15916825  0.02926718  0.08044716]
 [ 0.03796077  0.13057395  0.19755004]]
[[  7.72680961e-03   4.84951516e-02  -9.04411943e-03   2.46891708e-02
    3.38111565e-02   3.38111565e-02   3.38111565e-02   3.38111565e-02]
 [  4.44176700e-03   2.84723449e-03  -4.66745197e-03  -4.40795988e-03
    1.04302424e-02   1.04302424e-02   1.04302424e-02   1.04302424e-02]
 [  1.88642837e-02   2.92262598e-02  -7.96808311e-03  -3.26671735e-04
    5.52621022e-02   5.52621022e-02   5.52621022e-02   5.52621

[[ 0.03384706  0.01124021  0.05452879]
 [ 0.06020896  0.01833717  0.02012401]
 [ 0.08295534  0.00408081  0.03042579]
 [ 0.17752703  0.11055051  0.2097894 ]
 [ 0.26716166  0.22820548  0.07492015]
 [ 0.08351286  0.11302167  0.27571823]
 [ 0.40881147  0.47813885  0.28428726]
 [ 0.45919796  0.42350278  0.32741625]
 [ 0.41509077  0.45936709  0.26431693]
 [ 0.44601791  0.44255862  0.84732938]
 [ 1.41824648  1.06619626  3.32402866]
 [ 0.30780632  0.5619944   1.31957786]
 [ 0.14115389  0.05273382  0.07431825]
 [ 0.15339337  0.03289759  0.07923826]
 [ 0.03787202  0.12787593  0.19047647]]
[[  7.78336368e-03   4.69569065e-02  -8.96616768e-03   2.36266891e-02
    3.38470603e-02   3.38470603e-02   3.38470603e-02   3.38470603e-02]
 [  4.74159924e-03   3.09893501e-03  -4.96006140e-03  -4.66802874e-03
    1.12402146e-02   1.12402146e-02   1.12402146e-02   1.12402146e-02]
 [  1.83672724e-02   2.84471628e-02  -8.33401839e-03  -5.46510389e-04
    5.45287947e-02   5.45287947e-02   5.45287947e-02   5.45287

[[ 0.03391808  0.0120313   0.05387779]
 [ 0.05920167  0.01911764  0.02087197]
 [ 0.08126705  0.0047388   0.03076262]
 [ 0.17263833  0.10945223  0.20327063]
 [ 0.25852189  0.22121093  0.07450751]
 [ 0.08400931  0.11225977  0.26808269]
 [ 0.40617852  0.47591352  0.28248455]
 [ 0.45714991  0.42321616  0.32663081]
 [ 0.41077846  0.46147668  0.27191314]
 [ 0.44479216  0.43985445  0.80497237]
 [ 1.36749362  1.01655055  3.50657224]
 [ 0.31837705  0.56899215  1.32022558]
 [ 0.13702878  0.05294695  0.07528912]
 [ 0.14865167  0.03627017  0.07838538]
 [ 0.03830868  0.12547911  0.18432801]]
[[  7.84898637e-03   4.56035266e-02  -8.91134077e-03   2.26694036e-02
    3.39180757e-02   3.39180757e-02   3.39180757e-02   3.39180757e-02]
 [  5.02909655e-03   3.35091729e-03  -5.23984261e-03  -4.90899725e-03
    1.20313025e-02   1.20313025e-02   1.20313025e-02   1.20313025e-02]
 [  1.79427510e-02   2.77757183e-02  -8.65001123e-03  -7.34625875e-04
    5.38777869e-02   5.38777869e-02   5.38777869e-02   5.38777

[[ 0.03401478  0.01280209  0.05329606]
 [ 0.05831427  0.01987264  0.02159411]
 [ 0.07976499  0.00541328  0.03110724]
 [ 0.16817382  0.108535    0.1973644 ]
 [ 0.25066651  0.2149205   0.07429272]
 [ 0.08471145  0.11157494  0.26096199]
 [ 0.40400913  0.4739834   0.28124374]
 [ 0.45553893  0.42312751  0.32614728]
 [ 0.40755805  0.46324754  0.27858161]
 [ 0.44396373  0.43774055  0.76786903]
 [ 1.31433909  0.96881042  3.67665884]
 [ 0.3276001   0.57493649  1.32149053]
 [ 0.1334077   0.05336275  0.07613757]
 [ 0.14456291  0.039371    0.07774266]
 [ 0.03896195  0.1233388   0.17888795]]
[[  7.91995859e-03   4.44010994e-02  -8.87450081e-03   2.17985543e-02
    3.40147769e-02   3.40147769e-02   3.40147769e-02   3.40147769e-02]
 [  5.30354842e-03   3.60263220e-03  -5.50626266e-03  -5.13110491e-03
    1.28020877e-02   1.28020877e-02   1.28020877e-02   1.28020877e-02]
 [  1.75721967e-02   2.71913599e-02  -8.92416671e-03  -8.97936399e-04
    5.32960563e-02   5.32960563e-02   5.32960563e-02   5.32960